<a href="https://colab.research.google.com/github/marcociav/lango/blob/master/model-building/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lango - Model Training

## Imports

### Packages

In [ ]:
import pandas as pd

### Data

In [ ]:
df = pd.read_csv('data/sentences.csv', sep='\t', names=["id", "lan_code", "sentence"])

## Data Cleaning

In [ ]:
df.lan_code.unique()

array(['cmn', 'deu', 'rus', 'fra', 'eng', 'jpn', 'spa', 'ita', 'kor',
       'vie', 'nld', 'epo', 'por', 'tur', 'heb', 'hun', 'ell', 'ind',
       'ara', 'arz', 'fin', 'bul', 'yue', 'swe', 'ukr', 'bel', 'que',
       'ces', 'swh', 'nno', 'wuu', 'nob', 'zsm', 'est', 'kat', 'pol',
       'lat', 'urd', 'sqi', 'isl', 'fry', 'afr', 'ron', 'fao', 'san',
       'bre', 'tat', 'yid', 'uig', 'uzb', 'srp', 'qya', 'dan', 'pes', nan,
       'slk', 'eus', 'cycl', 'acm', 'tgl', 'lvs', 'kaz', 'hye', 'hin',
       'lit', 'ben', 'cat', 'bos', 'hrv', 'tha', 'orv', 'cha', 'mon',
       'lzh', 'scn', 'gle', 'mkd', 'slv', 'frm', 'glg', 'vol', 'ain',
       'jbo', 'tok', 'ina', 'nds', 'mal', 'tlh', 'roh', 'ltz', 'oss',
       'ido', 'gla', 'mlt', 'sco', 'ast', 'jav', 'oci', 'ile', 'ota',
       '\\N', 'xal', 'tel', 'sjn', 'nov', 'khm', 'tpi', 'ang', 'aze',
       'tgk', 'tuk', 'chv', 'hsb', 'dsb', 'bod', 'sme', 'cym', 'mri',
       'ksh', 'kmr', 'ewe', 'kab', 'ber', 'tpw', 'udm', 'lld', 'pms',
       'lad', 

In [ ]:
df = df.loc[~((df.lan_code.isna()) | (df.lan_code == '\\N'))]

In [ ]:
df.sentence = df.sentence.str.replace('[^\w\s]', '')

C:\Users\marco\AppData\Local\Temp\ipykernel_14592\1856176412.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.sentence = df.sentence.str.replace('[^\w\s]', '')


In [ ]:
df = df[~(df.sentence == '')]